In [ ]:
#Imports
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.utils import to_categorical
from google.colab import drive
from sklearn.model_selection import StratifiedKFold

print("Module Imports succesfully executed")

Module Imports succesfully executed


In [ ]:
#Mount the drive
drive.mount('/content/drive')
dir = "/content/drive/My Drive/Colab Notebooks/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Pre-processing

#Loading data
columns = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class']
df = pd.read_csv(dir+"nursery.data", header=None, index_col=None)
df.columns = columns

#Dropping minority classes
df = df.drop(df[df['class']=='recommend'].index)
df = df.drop(df[df['class']=='very_recom'].index)

#Changing to numerical and normalizing
df['parents'] = df['parents'].replace('usual', 0)
df['parents'] = df['parents'].replace('pretentious', 1)
df['parents'] = df['parents'].replace('great_pret', 2)
df['parents'] = df['parents']/2

df['has_nurs'] = df['has_nurs'].replace('proper', 0)
df['has_nurs'] = df['has_nurs'].replace('less_proper', 1)
df['has_nurs'] = df['has_nurs'].replace('improper', 2)
df['has_nurs'] = df['has_nurs'].replace('critical', 3)
df['has_nurs'] = df['has_nurs'].replace('very_crit', 4)
df['has_nurs'] = df['has_nurs']/4

df['form'] = df['form'].replace('complete', 0)
df['form'] = df['form'].replace('completed', 1)
df['form'] = df['form'].replace('incomplete', 2)
df['form'] = df['form'].replace('foster', 3)
df['form'] = df['form']/3

df['children'] = df['children'].replace('more', 4)
df['children'] = pd.to_numeric(df['children']).sub(1)
df['children'] = (df['children'])/3

df['housing'] = df['housing'].replace('convenient', 0)
df['housing'] = df['housing'].replace('less_conv', 1)
df['housing'] = df['housing'].replace('critical', 2)
df['housing'] = df['housing']/2

df['finance'] = df['finance'].replace('convenient', 0)
df['finance'] = df['finance'].replace('inconv', 1)

df['social'] = df['social'].replace('nonprob', 0)
df['social'] = df['social'].replace('slightly_prob', 1)
df['social'] = df['social'].replace('problematic', 2)
df['social'] = df['social']/2

df['health'] = df['health'].replace('priority', 0)
df['health'] = df['health'].replace('recommended', 1)
df['health'] = df['health'].replace('not_recom', 2)
df['health'] = df['health']/2

#Class is not normalized
df['class'] = df['class'].replace('spec_prior', 0)
df['class'] = df['class'].replace('priority', 1)
df['class'] = df['class'].replace('not_recom', 2)


print("Module Pre-processing succesfully executed")

Module Pre-processing succesfully executed


In [ ]:
#Train the model and compute the accuracy with a 10-fold cv
skf = StratifiedKFold(n_splits=10)
X = df.iloc[:, :len(df.columns)-1].to_numpy()
y = df.iloc[:, len(df.columns)-1].to_numpy()
total_acc_relu = 0
total_acc_sig = 0
total_acc_soft = 0

for train_index, test_index in skf.split(X, y):
  model_relu = Sequential([
                Dense(16, activation='relu', input_shape=(8,)),
                Dense(32, activation='relu'),
                Dense(3, activation='softmax')
  ])
  model_sig = Sequential([
                Dense(16, activation='sigmoid', input_shape=(8,)),
                Dense(32, activation='sigmoid'),
                Dense(3, activation='softmax')
  ])
  model_soft = Sequential([
                Dense(16, activation='softplus', input_shape=(8,)),
                Dense(32, activation='softplus'),
                Dense(3, activation='softmax')
  ])

  model_relu.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  model_sig.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  model_soft.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  model_relu.fit(X[train_index], y[train_index], epochs=10)
  model_sig.fit(X[train_index], y[train_index], epochs=10)
  model_soft.fit(X[train_index], y[train_index], epochs=10)
  
  test_loss, test_acc = model_relu.evaluate(X[test_index],  y[test_index], verbose=2)
  total_acc_relu += test_acc
  test_loss, test_acc = model_sig.evaluate(X[test_index],  y[test_index], verbose=2)
  total_acc_sig += test_acc
  test_loss, test_acc = model_soft.evaluate(X[test_index],  y[test_index], verbose=2)
  total_acc_soft += test_acc

print("A acurácia com o 10-fold cross-validation usando ReLU foi: "+"{:2.2%}".format(total_acc_relu/10))
print("A acurácia com o 10-fold cross-validation usando Sigmoid foi: "+"{:2.2%}".format(total_acc_sig/10))
print("A acurácia com o 10-fold cross-validation usando Soft Plus foi: "+"{:2.2%}".format(total_acc_soft/10))



print("Module Train & Test succesfully executed")

Epoch 1/10
356/356 [==============================] - 0s 1ms/step - loss: 0.6164 - accuracy: 0.7510
Epoch 2/10
356/356 [==============================] - 0s 1ms/step - loss: 0.2250 - accuracy: 0.9260
Epoch 3/10
356/356 [==============================] - 0s 1ms/step - loss: 0.1574 - accuracy: 0.9396
Epoch 4/10
356/356 [==============================] - 0s 1ms/step - loss: 0.1332 - accuracy: 0.9474
Epoch 5/10
356/356 [==============================] - 0s 1ms/step - loss: 0.1230 - accuracy: 0.9501
Epoch 6/10
356/356 [==============================] - 0s 1ms/step - loss: 0.1117 - accuracy: 0.9557
Epoch 7/10
356/356 [==============================] - 0s 1ms/step - loss: 0.1045 - accuracy: 0.9581
Epoch 8/10
356/356 [==============================] - 0s 1ms/step - loss: 0.0975 - accuracy: 0.9609
Epoch 9/10
356/356 [==============================] - 0s 1ms/step - loss: 0.0898 - accuracy: 0.9647
Epoch 10/10
356/356 [==============================] - 0s 1ms/step - loss: 0.0829 - accuracy: 0.9667

(12630, 8)